In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
df = pd.read_csv("data/all_data.csv")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_70504\3684764311.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all_data.csv")


In [4]:
df

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 42,Unnamed: 43,Unnamed: 39,Unnamed: 40
0,0,D1,2003-08-01,NaN,Bayern Munich,Ein Frankfurt,3.0,1.0,H,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,F1,2003-08-01,NaN,Lille,Lyon,1.0,0.0,H,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,F1,2003-08-02,NaN,Auxerre,Nice,1.0,2.0,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,F1,2003-08-02,NaN,Guingamp,Marseille,0.0,1.0,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,D1,2003-08-02,NaN,Hamburg,Hannover,0.0,3.0,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37784,37784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37785,37785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37786,37786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37787,37787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", 
                    "B365H", "B365D", "B365A",
                    "IWH", "IWD", "IWA",
                    "WHH", "WHD", "WHA",
                    "BbAH","BbAHh", "BbMxAHH", "BbAvAHH", "BbMxAHA", "BbAvAHA",
                    "GBAH", "GBAHH", "GBAHA",
                    "LBAH", "LBAHH", "LBAHA", "AHh", 
                    "B365AH", "B365AHH", "B365AHA"
                    ]
df = df[features_to_keep]

In [6]:
name_conversions = {
'St Etienne': 'Saint-Etienne',
'Ajaccio GFCO': 'Ajaccio', 
'Ath Madrid': 'Atletico',
'Ein Frankfurt': 'Frankfurt',
'Espanol': 'Espanyol',
'La Coruna': 'Depor',
'Nurnberg': 'Nuernberg',
'M\'gladbach': 'Gladbach',
'Bayern Munich': 'Bayern',
'Greuther Furth': 'Fuerth',
'Sp Gijon': 'Gijon',
'FC Koln': 'Koeln',
'Ath Bilbao': 'Bilbao',
'Hansa Rostock': 'Rostock',
'Fortuna Dusseldorf': 'Duesseldorf',
'Schalke 04': 'Schalke',
'Werder Bremen': 'Werder',
'Vallecano': 'RayoVallecano',
'Kaiserslautern': 'Lautern',
"Nott'm Forest": 'Forest',
'Holstein Kiel': 'Holstein',
'Munich 1860': 'Munich'
}

In [7]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))


In [8]:

club_elo_base_url = "http://api.clubelo.com/"
name_mismatches = []
exceptions = []
for team_name in all_teams:
    try:
        if exists("elo/" + team_name + ".csv"):
            continue
        team_name_url = name_conversions.get(team_name, team_name).replace(" ", "")
        team_data = requests.get(club_elo_base_url + team_name_url, stream=True)
        byte_data = team_data.content
        if team_name == "Nott'm Forest":
            a = 1
        if len(byte_data) == 38 or team_data.status_code != 200:
            # means that only header was returned, so team was not found
            name_mismatches.append(team_name)
            continue
        with open("elo/" + team_name + ".csv", mode='wb') as f:
            f.write(team_data.content)
    except:
        exceptions.append(team_name)
        continue
print("Name mismatches: ", name_mismatches)
print("Exceptions: ", exceptions)

Name mismatches:  ['Munich 1860', 'Evian Thonon Gaillard', 'Gimnastic', 'Arles']
Exceptions:  []


In [9]:
import datetime
from functools import partial

# for team in all_teams:
#     df.loc[(df["HomeTeam"] == team) | (df["AwayTeam"] == team)]
# # df = df.apply(add_elo_feature, axis=1)
# df.to_csv("data/all_data_with_elo.csv")
# df

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
        
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("data/all_data_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,GBAHH,GBAHA,LBAH,LBAHH,LBAHA,B365AH,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO
0,D1,2003-08-01,Bayern Munich,Ein Frankfurt,3.0,1.0,H,17.0,6.0,NaN,...,2.00,1.80,-1.75,1.80,2.05,NaN,NaN,NaN,1859.379272,1593.249268
1,F1,2003-08-01,Lille,Lyon,1.0,0.0,H,NaN,NaN,NaN,...,1.85,1.95,NaN,NaN,NaN,NaN,NaN,NaN,1612.968018,1726.539795
2,F1,2003-08-02,Auxerre,Nice,1.0,2.0,A,NaN,NaN,NaN,...,2.00,1.80,NaN,NaN,NaN,-0.75,2.050,1.850,1702.604858,1611.196045
3,F1,2003-08-02,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,NaN,...,1.90,1.90,NaN,NaN,NaN,0.00,1.925,1.975,1685.016113,1665.625732
4,D1,2003-08-02,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,NaN,...,1.75,2.10,-1.00,2.05,1.80,-0.75,1.800,2.100,1718.566284,1649.805298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37776,F1,2024-10-06,Reims,Montpellier,4.0,2.0,H,18.0,10.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.780,2.030,1633.626221,1645.806641
37777,SP1,2024-10-06,Sevilla,Betis,1.0,0.0,H,14.0,11.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.700,2.100,1676.242676,1709.259521
37778,SP1,2024-10-06,Sociedad,Ath Madrid,1.0,1.0,D,16.0,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.890,2.010,1766.551880,1828.522095
37779,F1,2024-10-06,Strasbourg,Lens,2.0,2.0,D,8.0,12.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.810,2.090,1608.732544,1719.138184


In [68]:
df["Date"][10000] > '2024-10-01'

False